## $\S$2. LDA

### 2.1 Mixed-Membership模型

#### 一. 何为Mix-Membership

1. 此前我们介绍了K-Means和EM, 分别是Hard assiagnment和Soft assignment. 而无论是硬分配还是软分配, 最终的目的都是找到文档所属的"一个主题". 而一篇文章, 往往涉及多个主题.此时Mixed Membership就是对文章进行多主题的分类模型  
2. 文章表示(Representation)-词袋模型  
 硬分配和软分配下, 文章被表示为一个TF-IDF向量(词频逆文档), 其长度等于单词表长度. 而在Mixed-Membership模型下, 使用Bag-of-Words进行文档表示. 词袋的意思是: 设想我们把一篇文章中的词抽出来放到一个袋子中, 然后摇晃这个袋子使得单词乱序的放在袋子中.因此, 词袋模型有以下特性:  
  1. 文档 $x_i$ 由一个list表示, 列表中为"文档中出现的单词".这些单词在列表中的顺序和在文档中的出现顺序可能不一致. 且文档中某个单词每出现一次, 就在词袋中加入一次这个单词. 因此一个单词可能在词袋列表中出现多次. 称之为"multiset"   
  2. 下图为文档$x_i$的表示图: 
  <img src="../../img/wordbag.png" height="70%" width="70%">
3. Mix-Membership的先验概率   
  1. 软分配下, 文章$x_i$所属cluster的先验概率为 $P\left( { z }_{ i }=k \right) ={ \pi  }_{ k }$   
  2. Mix-Membership下文章$x_i$所属cluster的先验概率分两步:  
    1. 确定每个topic下的词向量模型, 向量中记录(单词和其出现的概率).  
    eg:如下图, 主题science下"expirement"出现的概率为0.1,"test"出现的概率为0.08   
    2. 根据topic的词向量, 计算词袋模型表示的文档$x_i$, 在每个topic的词分布下, 出现$x_i$的似然概率  
    <img src="../../img/priorbow2.png" height="90%" width="90%">  
  
  
#### 二. LDA与Mix-Membership
1. LDA的目标   
 LDA是要找到一个文档对应的一些列主题
2. 数学符号  
  1. $Z_{iw}$ : 单词w(文档i中的), 在某个主题下的得分
  2. $\pi_{i}=[\pi_{i1},\pi_{i2}...\pi_{ik}]$ : 文档在某个topic下的prevalence    
3. 贝叶斯推断  
 贝叶斯推断不与最大似然估计一样. 最大似然估计(MLE)对参数估计了一个确定值, 这个参数值使得概率值 $P(事件发生|该分布下)$ 
最大.  
 而贝叶斯推断为最大后验估计(MPE-maximum posterior estimazation). 因为贝叶斯推断对参数估计出一组值, 然后对这些值赋予一个权重. 最终的推断结果就是这些值得加权平均  
 LDA被认作是一个"贝叶斯模型"



### 2.2 贝叶斯推断与Gibbs抽样
#### 一. Gibbs抽样
1. Gibbs sampling是一种迭代式的硬分配算法. 这种"硬分配"不同于K-Means.   
 K-Means是从一个等式中求解$centroId=min[dist(x^i,allcentroId)]$. 而这里, 我们的模型参数和被"硬分配"的变量值是通过Gibbs抽样产生的一系列值
2. 贝叶斯模型中, 我们要关注的是, 多次迭代每次产生的模型参数自身的概率值, 从而对最终的模型参数进行加权估计. 这称之为"joint model probility"  
 在迭代过程图中(x:迭代次数,y:该次估计值为真实值的概率), 模型参数从初始值收敛到某一块区域.  之后在这区域内上下波动. 此时认为"联合模型概率"已经收敛. 因为gibbs抽样是随机的, 所以这些估计值不会收敛到一个点, 而是收敛到一个区域内.最终gibbs抽样产生的硬分配加权结果是"正确贝叶斯估计的"
 <img src="../../img/gibbs1.png" id = "gibbs1">
 
#### 二. LDA中标准Gibbs采样过程
#### 第一步: 随机分配文章i中单词w属于的topic为k, 其概率: ${ r }_{ iwk }$  
  假设我们已获得每个topic的单词概率表, 重新随机硬分配 $z_{iw}=k$ .即重新分配文档 $i$ 中某个单词$w$ 是属于主题 $z$ 的.   
  1. 假设随机分配该单词来自于主题2, 即$z_{iw}=2$的概率${ r }_{ iw2 }=\frac { { \pi  }_{ i2 } * p("EEG"|{ z }_{ iw }=2) }{ \sum _{ j=1 }^{ \pi  }{ { \pi  }_{ ij } * p("EEG"|{ z }_{ iw }=j) }  } $    
  2. 重复此过程在文档中的所有单词  
 [注]: 此过程, 就像高斯混合的第一步: 根据已知的k个高斯分布, 计算该文档属于分布k的概率  
 
#### 第二步: 随机分配文档i属于topic k, 其概率为: ${\pi}_{ik}$  
 我们统计文档i共多少次(带第一步算出的权值)被y硬分配给了主题k, 从而计算该文档属于主题k的概率
#### 第三步: 将前两步应用在文本集中的所有文档上  
#### 第四步: 随机重新分配"topic词典"中的单词概率
 根据所有文档中单词w的被硬分配给$z_{iw}=k$的次数, 计算单词w在topic=k上存在的概率, 重新形成"topic词典"的概率分布. 同样这个"topic词典"的计算也是通过贝叶斯估计加权而来  
 <img src="../../img/gibbs2.png">

#### 三. Gibbs停止条件
重复进行1~4步, 直到plot出的联合模型概率图达到最大区域. <a href="#gibbs1" target="_self">即此图</a>

### 2.3 collapse gibbs采样
#### 一. 总览
1. collapse gibbs sampling :   
 一些特殊的LDA模型中, 我们只随机分配 $z_{iw}$ , 不再随机分配 $r_{iw}$ 和 $\pi_{ik}$.也不去计算toip 词典的单词概率和Document所属topic的概率  
2. 我们要做的只是: 基于所有文档中其他单词的$z_{jv}$, 来分配本文档的$z_{iw}$
3. clollapse gibbs 采样概念图如下, 隐去了主题词典概率表和文档对应的主题概率
<img src="../../img/collapse1.png" width="80%" height="80%">

#### 二. collapse gibbs采样过程
1. 初始化所有词频统计指标  
  1. 我们随机对每个文档中的单词分配topic.  
  2. 然后分别对每个文档分别统计每个toic下有多少个单词(local):local statistics  
  和在全部文档集统计每个toic下有多少个单词(global): global statistics  
<img src="../../img/initialcollapse.png" width="90%" height="90%">

2. 对于某个文档i, 随机删除其包含的一个单词w. 此处我们删除单词"dynamic". 使得local的主题次品统计和global的主题词频统计减1.  
 而后我们要根据现有文档集的词频统计,重新对文档i中的单词"dynamic"分配主题. 因此善生两个问题:  
  1. 文档i属于主题k的比率 : $p({ z }_{ i }=k)=\frac { { n }_{ ik }+\alpha  }{ { N }_{ i }-1+K\alpha  } $  
    1. ${ n }_{ ik }$ : 文档i中被分配给主题k的单词个数
    2. ${ N }_{ i }$-1 : 文档i一共有多少个单词 (已删除单词dynamic)
    3. $\alpha $ : smooth param
  2. 主题k有多么需要单词 $$\frac { { m }_{ dynamic,k }\quad +\gamma  }{ \sum _{ w\in V }^{  }{ { m }_{ w,k } } \quad +V\gamma  } $$ 
    1. ${ m }_{ dynamic,k }$ : 所有文档中, "dynamic"被分配给主题k的次数
    2. $\sum _{ w\in V }^{  }{ { m }_{ w,k } } $ : 所有文档中, 所有被分配给主题k的单词个数
    3. $V$ : 单词表长度
    4. $\gamma$ : smooth param
    <img src="../../img/decollapse.png" width="110%" height="110%">

3. 根据第二步得出的两个指标 : "文档i有多么属于主题k"和"主题k有多么需要单词w". 将他们相乘, 得到该文档i中的单词w, 分别属于对应主题的概率
<img src="../../img/decollapse2.png" width="100%" height="100%">

#### 三. collapse sampling结果使用
1. collapse sampling最终结果是对每个文档中的单词w分配了最佳的topic, 即 $z_{iw}$. 因此, 可以借此推断两个事情: 
  1. topic在这些文档中的分布
  2. 每个文档的向量表示(元素为所属topic概率)